In [35]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [36]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()
session.get_current_schema().replace('"','')

'INSIGHT_SPCS_SCHEMA'

In [352]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier,RandomForestRegressor
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [353]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv")

df_filtered = df[df['SALARY']!='low']
snow_df = session.create_dataframe(df_filtered)
# snow_df = session.create_dataframe(df)


snow_df = snow_df.drop(['Unnamed: 0','DEPARTMENT'])
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
test_df = apply_label_encoding(test_df)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [343]:
input_data_frame.limit(2).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |0.11                  |0.88               |7                 |272                     |4                     |0                |1       |0                        |
------------------------------------------------------------------------------------------------------------------------

In [359]:
pipeline = RandomForestRegressor(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)
pred_snow_df = pipeline.predict(test_df)
pred_snow_df.columns

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


['SALARY',
 'SATISFACTION_LEVEL',
 'LAST_EVALUATION',
 'NUMBER_PROJECT',
 'AVERAGE_MONTLY_HOURS',
 'TIME_SPEND_COMPANY',
 'WORK_ACCIDENT',
 'LEFT',
 'PROMOTION_LAST_5YEARS',
 'OUTPUT_SALARY']

In [360]:
feature_columns = input_data_frame.columns
sf_current = pred_snow_df
sf_reference = pred_snow_df
target_column  = "SALARY"
prediction_column = "OUTPUT_SALARY"
# predict_probability_columns = [ '"""PREDICT_PROBA_high"""','"""PREDICT_PROBA_medium"""']
# predict_probability_columns = ['"""PREDICT_PROBA_0.0"""',
#  '"""PREDICT_PROBA_2.0"""']
drift_configs = {
        "project_id" : "asvw4g124-124sdvrgwef",
        "feature_columns": feature_columns,
        "current_dataset" : current_df,
        "reference_dataset" : reference_df,
        "model_name" : "HR_CHURN_MODEL",
        "problem_type" : "regression",  # classification or regression
        "target_column" : "SALARY",
        "prediction_column" : prediction_column,
        # "predict_probability_columns" : predict_probability_columns,
        "version_name" : "V2",
        "alerts_configurations" : [
            {
                "parameter": "performance_drift_score",
                "threshold_range": [[0.5, 1,'Red'],[0.5, 1,'Amber'],[0.5, 1,'Green']]
            },
            {
                "parameter": "drift_score",
                "threshold_range": [[0.5, 1,'Red'],[0.5, 1,'Amber'],[0.5, 1,'Green']]
            }
        ]
    }

In [363]:
execute_regression_drift(drift_configs,
                             sf_current,
                             sf_reference)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function explained_variance_score.<locals>.explained_variance_score_anon_sproc at 0x7f734f705d30>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <function explained_variance_score.<locals>.explained_variance_score_anon_sproc at 0x7f734f74d8b0>. Proceeding without cr

{'metrics': [{'regression_quality_metrics': {'current': {'explained_variance_score': 0.17858650301441192,
     'mean_absolute_percentage_error': 462391990082562.0,
     'mean_squared_error': 0.10988715626568027,
     'mean_absolute_error': 0.22966124427348916,
     'r2_score': 0.1741421112267515,
     'root_mean_square_error': 0.41730338032030306},
    'reference': {'explained_variance_score': 0.1785865030144118,
     'mean_absolute_percentage_error': 462391990082562.06,
     'mean_squared_error': 0.10988715626568027,
     'mean_absolute_error': 0.2296612442734892,
     'r2_score': 0.1741421112267515,
     'root_mean_square_error': 0.41730338032030306}}},
  {'prediction_vs_actual': {'current': {'x_axis': [1.0,
      1.0,
      1.0,
      1.0,
      1.0,
      1.0,
      1.0,
      0.0,
      1.0,
      1.0,
      0.0,
      1.0,
      1.0,
      1.0,
      1.0,
      1.0,
      1.0,
      1.0,
      0.0,
      1.0,
      0.0,
      1.0,
      1.0,
      1.0,
      1.0,
      1.0,
     

In [361]:
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, try_cast, when, cast
from sklearn.datasets import load_diabetes
import json

from snowflake.ml.modeling.metrics import explained_variance_score, \
                    mean_absolute_percentage_error ,\
					mean_squared_error, \
					mean_absolute_error, \
					r2_score


In [362]:

def get_regression_metrics(sf_df,true_cn,pred_cn):
    import math
    performance_metrics = {}
    performance_metrics['explained_variance_score'] = explained_variance_score(df=sf_df,
                                                                               y_true_col_names=true_cn,
                                                                               y_pred_col_names=pred_cn)

    performance_metrics['mean_absolute_percentage_error'] = mean_absolute_percentage_error(df=sf_df,
                                                                                            y_true_col_names=true_cn,
                                                                                            y_pred_col_names=pred_cn)

    performance_metrics['mean_squared_error'] = mean_squared_error(df=sf_df,
                                                                    y_true_col_names=true_cn, 
                                                                    y_pred_col_names=pred_cn)

    performance_metrics['mean_absolute_error'] = mean_absolute_error(df=sf_df,
                                                                        y_true_col_names=true_cn, 
                                                                        y_pred_col_names=pred_cn)

    performance_metrics['r2_score'] = r2_score(df=sf_df,y_true_col_name=true_cn, y_pred_col_name=pred_cn)

    performance_metrics['root_mean_square_error'] = math.sqrt(performance_metrics['r2_score'])

    return performance_metrics


def get_list_of_bins(total_rows):
    total_bins = 150
    bin_indexs = [] ; count = 0
    if total_rows > 150 :
        for i in range(0,total_rows,int(total_rows/total_bins)):
            bin_indexs.append(i)
            count += 1
            if count > 150 :
                break
    else :
        return [i for i in range(total_rows)]
    return bin_indexs

def get_binned_sf_dataframe(sf_df,plot_bins):
    from snowflake.snowpark.functions import row_number, lit, col
    from snowflake.snowpark.window import Window

    window_spec = Window.order_by(lit(1))
    df_with_row_numbers = sf_df.with_column("row_number",row_number().over(window_spec))
    temp_df = df_with_row_numbers.filter(col("row_number").isin(plot_bins)).drop("row_number")
    return temp_df

def get_prediction_vs_actual_data(sf_df,true_cn,pred_cn):
    regression_plot = {}
    regression_plot["x_axis"] = sf_df.select(true_cn).to_pandas().squeeze().tolist()
    regression_plot["y_axis"] = sf_df.select(pred_cn).to_pandas().squeeze().tolist()
    return regression_plot

def get_regression_plot_in_time(sf_df,true_cn,pred_cn,bins):
    regression_in_time_plot = {}
    target_plot = {} ; prediction_plot = {}
    target_plot['x_axis'] = bins[:-1]
    target_plot['y_axis'] = sf_df.select(true_cn).to_pandas().squeeze().tolist()
    prediction_plot['x_axis'] = bins[:-1]
    prediction_plot['y_axis'] = sf_df.select(pred_cn).to_pandas().squeeze().tolist()
    regression_in_time_plot[true_cn] = target_plot
    regression_in_time_plot[pred_cn] = prediction_plot
    return regression_in_time_plot

def get_errors_plot(sf_df,true_cn,pred_cn,bins):
    error_plot = {}
    error_plot['x_axis'] = bins[:-1]
    error_plot['y_axis'] = sf_df.with_column("difference",col(pred_cn)-col(true_cn)) \
                            .select("difference").to_pandas().squeeze().tolist()
    return error_plot

def get_error_normality_plot(sf_df,true_cn,pred_cn,bins):
    import numpy as np
    import scipy.stats as stats
    error_normality_plots = {}
    errors = sf_df.with_column("difference",col(pred_cn)-col(true_cn)) \
                            .select("difference").to_pandas().squeeze().tolist()
    sorted_errors = np.sort(np.array(errors))
    n = len(sorted_errors)

    theoretical_quantiles = stats.norm.ppf((np.arange(1,n+1) - 0.5)/n)
    error_normality_plots['theoretical_quantiles'] = theoretical_quantiles.tolist()
    error_normality_plots['prediction_errors'] = sorted_errors.tolist()
    return error_normality_plots


def execute_regression_drift(drift_configs,
                             sf_current,
                             sf_reference):
    
    ####################################Regression problem ########################################

    metrics = []
    target_column = drift_configs['target_column']
    prediction_column = drift_configs['prediction_column']

    ## bins
    plot_bins = get_list_of_bins(sf_current.count())

    # binned dataframes
    binn_sf_current = get_binned_sf_dataframe(sf_current,plot_bins)
    binn_sf_reference = get_binned_sf_dataframe(sf_current,plot_bins)

    # performance metrics
    regression_quality_metrics = {}
    regression_quality_metrics['current'] = get_regression_metrics(sf_current,target_column,prediction_column)
    regression_quality_metrics['reference'] = get_regression_metrics(sf_reference,target_column,prediction_column)
    metrics.append({
        "regression_quality_metrics" : regression_quality_metrics
    })
    # print(regression_quality_metrics)

    # Prediction vs Actual
    prediction_vs_actual = {}
    prediction_vs_actual['current'] = get_prediction_vs_actual_data(binn_sf_current,target_column,prediction_column)
    prediction_vs_actual['reference'] = get_prediction_vs_actual_data(binn_sf_reference,target_column,prediction_column)
    metrics.append({
        "prediction_vs_actual" : prediction_vs_actual
    })
    # print(prediction_vs_actual)

    # Prediction vs Actual in time
    prediction_vs_actual_in_time = {}
    prediction_vs_actual_in_time['current'] = get_regression_plot_in_time(binn_sf_current,target_column,prediction_column,plot_bins)
    prediction_vs_actual_in_time['reference'] = get_regression_plot_in_time(binn_sf_reference,target_column,prediction_column,plot_bins)
    metrics.append({
        "prediction_vs_actual_in_time" : prediction_vs_actual_in_time
    })
    # print(prediction_vs_actual_in_time)

    # Errors predicted vs Actual in time
    errors_plot = {}
    errors_plot['current'] = get_errors_plot(binn_sf_current,target_column,prediction_column,plot_bins)
    errors_plot['reference'] = get_errors_plot(binn_sf_reference,target_column,prediction_column,plot_bins)
    metrics.append({
        "errors_plot" : errors_plot
    })
    # print(errors_plot)

    ## Error normality
    errors_normality_plot = {}
    errors_normality_plot['current'] = get_error_normality_plot(binn_sf_current,target_column,prediction_column,plot_bins)
    errors_normality_plot['reference'] = get_error_normality_plot(binn_sf_reference,target_column,prediction_column,plot_bins)
    metrics.append({
        "errors_normality_plot" : errors_normality_plot
    })
    # print(errors_normality_plot)

    return {
        "metrics" : metrics
    }


In [46]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="Testmode02092024",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="Notebook",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_TESTMODE02092024' registered successfully."